In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [16]:
import joblib

# Load the DataFrame from the Joblib file
data = joblib.load('my_dataframe.joblib')


In [17]:
X = data.drop(columns=['SalePrice'])
y = data['SalePrice']

In [22]:
# Encoding categorical columns 
categorical_columns=data.select_dtypes(include=['object']).columns.tolist()
X=data.drop(columns=categorical_columns)
b_c=['Neighborhood', 'KitchenQual', 'BsmtQual', 'GarageType', 'CentralAir', 'Exterior2nd', 'HouseStyle', 'Exterior1st', 'BsmtFinType1', 'BsmtExposure']
best_categorical=data[b_c]
X = pd.concat([X, best_categorical], axis=1)
X=pd.get_dummies(X, columns=b_c)

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [25]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()      
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train,y_train)

LinearRegression()

In [27]:
y_pred = regressor.predict(X_test)
y_pred2=regressor.predict(X_train)

In [28]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.999999999870844

In [29]:
r2_score(y_train, y_pred2)

1.0

In [30]:
from sklearn.metrics import mean_squared_error
# Calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")


Root Mean Squared Error (RMSE): 0.75


In [31]:
rmse2 = mean_squared_error(y_train, y_pred2, squared=False)
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

Root Mean Squared Error (RMSE): 0.75


In [32]:
mean_saleprice = data['SalePrice'].mean()
relative_rmse = (rmse / mean_saleprice) * 100
print("Relative RMSE (%):", relative_rmse)

Relative RMSE (%): 0.0004246688516257452


In [33]:
relative_rmse2 = (rmse2 / mean_saleprice) * 100
print("Relative RMSE (%):", relative_rmse2)

Relative RMSE (%): 5.1838563120457416e-14


In [ ]:
# Binary Encoding (for binary features)
data['CentralAir'] = data['CentralAir'].map({'Y': 1, 'N': 0})

In [ ]:
ord_c=['ExterQual','ExterCond', 'BsmtQual','BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual','GarageCond']

In [ ]:
# Define the quality mappings
quality_mapping = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1,'NA' : 0}

# Map the quality-related columns
data['ExterQual'] = data['ExterQual'].map(quality_mapping)
data['ExterCond'] = data['ExterCond'].map(quality_mapping)
data['BsmtQual'] = data['BsmtQual'].map(quality_mapping)
data['BsmtCond'] = data['BsmtCond'].map(quality_mapping)
data['HeatingQC'] = data['HeatingQC'].map(quality_mapping)
data['KitchenQual'] = data['KitchenQual'].map(quality_mapping)
data['FireplaceQu'] = data['FireplaceQu'].map(quality_mapping)
data['GarageQual'] = data['GarageQual'].map(quality_mapping)
data['GarageCond'] = data['GarageCond'].map(quality_mapping)

In [ ]:
def combine_rare_categories(df, column, threshold=15):
    # Get category counts
    category_counts = df[column].value_counts()
    
    # Define rare categories
    rare_categories = category_counts[category_counts < threshold].index
    
    # Replace rare categories with 'Other'
    df[column] = df[column].apply(lambda x: 'Other' if x in rare_categories else x)
    return df

columns_to_adjust = data.select_dtypes(include=['object']).columns.tolist()
columns_to_adjust.remove("Neighborhood")

for col in columns_to_adjust:
    data = combine_rare_categories(data, col)

In [ ]:
def target_based_encoding(df, target_col, cat_cols):
    """
    Apply target-based encoding to multiple categorical columns based on the mean of the target variable.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    target_col (str): The name of the target column (e.g., 'SalePrice').
    cat_cols (list): A list of categorical columns to encode.

    Returns:
    pd.DataFrame: DataFrame with the target-encoded columns.
    """
    df_encoded = df.copy()
    for col in cat_cols:
        # Step 1: Calculate the mean target value for each category in the column
        category_means = df.groupby(col)[target_col].mean()

        # Step 2: Rank categories by their mean target value and assign encoding
        category_ranks = category_means.rank().astype(int) - 1  # Rank starts from 0
        category_map = category_ranks.sort_values().astype(int).to_dict()

        # Step 3: Map the categories to their respective ranks (encoded values)
        df_encoded[col] = df[col].map(category_map)

    return df_encoded

In [ ]:
encoded_data = target_based_encoding(data, target_col='SalePrice', cat_cols=categorical_columns)